# Landsat Time Series Generation

This tutorial demonstrates how to use pee to create Landsat time series images and creating and exporting a LandTrendr fitted time series.

In [ ]:
import ee, sys, os, subprocess, rasterio
sys.path.append("..") # try to avoid this somehow; maybe by making pee an installed editable package so it's in the path already
from glob import glob

import numpy as np
import time_series as ts
import landsat as lxtools

ee.Initialize()

In [ ]:
# This is just a helper function to check on multiple export tasks, which are done throughout the tutorial
def check_tasks(task_list):
    statuses = [task.status() for task in task_list]
    for status in statuses:
        if 'start_timestamp_ms' in status.keys():
            runtime = (status['update_timestamp_ms'] - status['start_timestamp_ms'])/1000./60
        else:
            runtime = 0
        print(status['description'], status['state'], round(runtime, 2), 'min')
    return statuses

# Landsat Image Collection
Ceate an ee.ImageCollection of Landsat Surface Reflectance images given an area, time frame, and host of other options. This example provides some details on what some parameters are and how to change them, but in general the default settings are pretty good.

In [ ]:
# You can use a Geometry, Feature, or featureCollection for spatial filtering of the images
# This is just past to filterBounds
aoi = ee.FeatureCollection('TIGER/2018/Counties').filterMetadata('NAME', 'equals', 'Larimer')

# Start and end can be anything accepted by filterDate
# NOTE: end is exclusive but start, startdoy and enddoy are inclusive
start = '2018'
end = '2019'     # since end is exclusive, using '2019' as end will filter to only 2018 images

# If filtering a collection of multiple years using 'start' and 'end' you can use
# start day of year and end day of year parameters
# to filter to a time frame within each year.
startdoy = 152   # June 1st
enddoy = 273     # Sept 30th

**Cloud masking**  
Cloud masking here is a catch-all phrase for masking out clouds, snow, water, saturated pixels, etc, etc. The surface reflectance images come with several quality assurance bands to apply cloud masking through bit masks. The sr_mask function simplifies the various masking options. 

sr_mask calls pqa_mask which by default filters the pqa band for "clear" pixels with no water, shadows, clouds, or snow and which have not been filled. It also filters for pixels for pixels that have "confidence" values for a "low" chance of normal or cirrus clouds. You can change these options by specifying water=1 if you want only pixels that have water or water=None if you want to keep both water or non-water pixels. For confidence values you can give cloud_conf=2 if you want to accept pixels that have a 'medium' chance of being clouds.

sr_mask also filters for pixels in the valid data range of 0 to 10,000 for optical bands and which don't have radiometric saturation according the rad_qa band. Note that blue and coastal blue are often oversaturation, so this may filter more pixels than desired. In Landsat 5 some edge pixels don't have values for all bands, and these pixels are masked out by default as well. For Landsat 8, aerosols can be masked out too.

Below we bundle these options into a dictionary that is passed to the sr_mask function. See the sr_mask function, pqa_mask function and Landsat product guides for details on applying cloud masking.

In [ ]:
# Lets allow water and only use the confidence bands for cloud masking since the product guides
# recommend using either the cloud and clear switches or the confidence bands, not both.
mask_kwargs = {'water':None,
               'clear':None,
               'cloud':None,
               'cloud_conf':2,  # allow low and medium confidence clouds
               'cirrus_conf':1  # only low confidence cirrus filtering (only for L8)
              }

**Temporal Dark Outlier Mask**  
The F-mask algorithm is fairly effective at identifying clouds, snow and water, but often misses cloud shadows. Cloud shadows that are not masked out correctly can have problems particularly for mapping forest structure since shadows have a strong effect on the SWIR bands. To more be more rigorous in removing cloud shadows we can tack on the Temporary Dark Outlier Mask (TDOM) algorithm. This method uses a long time series of the pixel to identify outliers that are dark in SWIR and NIR. By default a morphological opening is also applied to this mask. To use TDOM the pixel level mean and standard deviation needs to be calculated across a fairly long time series, so it's best if precomputed stats images are used.

In [ ]:
# Load statistics images that were precalculated by GTAC for use with TDOM
# These images have bands of the mean and standard deviation for different landsat bins using
# data up to 2019 which is enough to calculate if the pixels in each image are outliers.
TDOMStats = ee.ImageCollection('projects/lcms-tcc-shared/assets/CS-TDOM-Stats/TDOM').mosaic()#.divide(10000) divide if using 0-1 imagery, don't if using unscaled images (0-10,000)

# select and rename the bands to use for TDOM so that the band names match the names of image collection we'll be making
mean_img = TDOMStats.select(['Landsat_nir_mean', 'Landsat_swir1_mean'], ['nir', 'swir1']) 
stddev_img = TDOMStats.select(['Landsat_nir_stdDev', 'Landsat_swir1_stdDev'], ['nir', 'swir1'])

# Make a dictionary of arguments to pass to TDOM including the mean and stddev images to use
# Since we're using the unscaled TDOMStats images we'll using a sum threshold of 3500 instead of the default 0.35
tdom_kwargs = {'mean_img':mean_img, 'stddev_img':stddev_img, 'sum_thresh':3500}

All of the parameters for setting up an image collection are fed into a single function that filters images from the given Landsat satellite collections, renames the bands, applies cloud masking, and harmonizes Landsat 8 to match Landsat 7.

In [ ]:
images = lxtools.sr_collection(
            aoi, start, end,        # Area, start date, and end date are required parameters
            startdoy,
            enddoy,
            sats=["LANDSAT_7", "LANDSAT_8"],
#             harmonize=True,        # This harmonizes L8 to match L5/7 since they have slightly different band widths, but people debate whether this is really needed
            rescale=False,         # Since we're using the unscaled (0-10k) TDOM stats we'll also not scale the images here
            cloud_cover=50,        # Filter out images that have greater than 50% cloud cover. Images with lots of cloud cover often have some unmasked clouds and cloud shadows
            slc_on=False,          # If true then only using L7 images when the scan line corrector was working (January 1998 - May 2003)
            tdom=True,
            exclude=['LE07_033032_20180612', 'LC08_034031_20180729'],          # You can provide a list of specific images to exclude by their system:index
            mask_kwargs=mask_kwargs,
            tdom_kwargs=tdom_kwargs
                 )

In [ ]:
# Check the number of images in the filtered collection
images.size().getInfo()

In [ ]:
# The first image is from June 2nd and the last is from Sept 30th
print(ee.Date(images.aggregate_min('system:time_start')).format('YYYY-MM-dd').getInfo())
print(ee.Date(images.aggregate_max('system:time_start')).format('YYYY-MM-dd').getInfo())

## Compositing and Spectral Indices 

### One year composite
Lets composite this one year summer image collection into a single image that should be representative of the conditions over summer. We'll do this using the medoid operation which is like a multidimensional median. First we'll filter to the shared set of optical bands.

In [ ]:
# Get the list of L7 optical bands
l7_opt = lxtools.__sr_dict['LANDSAT_7']['opt']
l7_opt

In [ ]:
images = images.select(l7_opt)

In [ ]:
# get the medoid using just a few bands, and add a band with day of year for the pixel selected from the collection
img = ts.medoid(images, med_bands = ['red', 'nir', 'swir1'], date_band=True)
img.bandNames().getInfo()

You can see that all bands were retained but only red, nir and swir1 were used in the medoid calculation. A 'date' band with the day of year was also added.

Let's calculate some spectral indices from this medoid composite. Since the medoid selects a single pixel from a specific date the band interrelationships are maintained so calculating spectral indices from the composite is still valid. This would not be the case if we used a different compositing method like the median or mean.

In [ ]:
# Let's calculate the NDVI, NDMI, and all tasseled cap indices
specs = lxtools.specixs(img, ixlist=['ndvi', 'ndmi', 'tcw', 'tcb'])#'tasseled_cap'])
specs.bandNames().getInfo()

### Annual Composites
If you need image composites over multiple years this process can be simplified with the annual composites function, which takes all of the same parameters and a compositing function.

In [ ]:
# Use several years a wider 
starty = 2015
endy = 2020   # end inclusive

# Bundle all the collection arguments from before into a single dictionary
# Note that it's only necessary to provide keyword arguments where the options we want are different from the defaults
coll_kwargs = {
    'rescale':False,
    'bands':['blue', 'green', 'red', 'nir', 'swir1', 'swir2'],
    'cloud_cover':50,
    'tdom':True,
    'mask_kwargs':mask_kwargs,
    'tdom_kwargs':tdom_kwargs,
    'exclude': ['LE07_033032_20180612', 'LC08_034031_20180729']  # In this case let's exclude a couple images we don't want using their system:index in their original collections
}

comps = ts.annual_composites(
            aoi, starty, endy, startdoy, enddoy,       # use the same spatial and day of year filters as before
            coll_func = lxtools.sr_collection,         # You can provide a different function for obtaining the collection if you want
            comp_func = ee.ImageCollection.median,     # This time lets just use the median function in GEE
            coll_kwargs=coll_kwargs,
            fill=True                                  # Fill years with no images after filtering with an empty image, this is needed for LandTrendr 
)

In [ ]:
# We have 6 images. 
print(comps.size().getInfo())

# One for each year from 2015 to 2020
print(comps.aggregate_array('year').getInfo())

**Annual composites with spectral indices**  
We can calculate spectral indices for each composite, but since we used the median which could be from different dates for each band, it's better to calculate spectral indices before compositing. Perhaps the easiest way to do this is by adding the spectral index calculation as the first step in the compositing function.

In [ ]:
def median_specix(imgs, ixlist=None):
    """ For the set of images in each year calculate spectral indices and return the annual median.
    """
    imgs = imgs.map(lambda i: lxtools.specixs(i, ixlist=ixlist))
    img = imgs.median()
    return img

comps = ts.annual_composites(
            aoi, starty, endy, startdoy, enddoy,
            coll_func = lxtools.sr_collection,
            comp_func = median_specix,
            coll_kwargs=coll_kwargs,
            comp_kwargs = {'ixlist':['ndvi', 'nbr']},   # we'll get these spectral indices before compositing
            fill=True
)

In [ ]:
# We still have an image for every year
print(comps.aggregate_array('year').getInfo())

# Now they're the median ndvi and nbr values
print(comps.first().bandNames().getInfo())

## Exporting Tiles
We generally work over large areas so when working with images locally, tiling is the often the most efficient way to store a set of images covering an area. We'll export the annual composites we just made in a set of Landsat Analysis Ready Data (ARD) tiles covering Larimer county. 

Here I use a previously stored copy of the Landsat ARD tiles that I've uploaded as a GEE asset, but you could use any GEE feature collection with polygons specifying different aois that you want to make exact exports for. The tile features have attributes that give the min and max coordinates of the tile in the CONUS Albers project (EPSG:5070), which is what's used for ARD.

In [ ]:
# Filter for ARD tiles intersecting the AOI
aoi = ee.FeatureCollection('TIGER/2018/Counties').filterMetadata('NAME', 'equals', 'Larimer')
tilesfc = ee.FeatureCollection("users/stevenf/other/conus_ard_grid_coords")
tilesfc = tilesfc.filterBounds(aoi)

Creating a separate composite set for each tile may reduce some unnecessary processing for GEE on the back end and speed up the export. So we'll create the composites within the tile exporting loop rather than just creating the composite set for the entire aoi. In general it's best to first do as much filtering of images as you can for a specific output, in this case filtering images to just the tile rather than the entire AOI.

In [ ]:
# Get a list of tiles to loop over
tilehvs = tilesfc.aggregate_array('hv').getInfo()
tilehvs

For ARD tiles the 'hv' attribute gives the horizonal cell number plus the vertical cell number. For example, '011008' is horizontal cell 011 and vertical cell 008.

In [ ]:
# Specify annual composite collection parameters

# time filtering
starty = 2018
endy = 2019 
startdoy = 152   # June 1st
enddoy = 273     # Sept 30th

# TDOM set up
TDOMStats = ee.ImageCollection('projects/lcms-tcc-shared/assets/CS-TDOM-Stats/TDOM').mosaic()
mean_img = TDOMStats.select(['Landsat_nir_mean', 'Landsat_swir1_mean'], ['nir', 'swir1']) 
stddev_img = TDOMStats.select(['Landsat_nir_stdDev', 'Landsat_swir1_stdDev'], ['nir', 'swir1'])
tdom_kwargs = {'mean_img':mean_img, 'stddev_img':stddev_img, 'sum_thresh':3500}

# Masking differences from defaults
mask_kwargs = {
    'water':None
}

# Collection options
coll_kwargs = {
    'rescale':False,
    'bands':['red', 'nir', 'swir1', 'swir2'],  # these are the only ones needed for TDOM and creating NDVI and NBR
    'cloud_cover':50,
    'tdom':True,
    'mask_kwargs':mask_kwargs,
    'tdom_kwargs':tdom_kwargs,
    'exclude': ['LE07_033032_20180612', 'LC08_034031_20180729']
}

# Compositing function
def median_specix(imgs):
    imgs = imgs.map(lambda i: lxtools.specixs(i, ixlist=['ndvi', 'nbr']))
    return imgs.median()

### Separate - one image per tile and year
This method is straighforward but slower to export overall (~7 min per tile, 22 min overall). The advantage is that image bands don't need to be rearranged after export.

In [ ]:
# Export one annual composite image for each tile and year
task_list = []
years = np.arange(starty, endy+1)
for hv in tilehvs:
    for year in years:
        print(hv, year)
        tile = ee.Feature(tilesfc.filterMetadata('hv', 'equals', hv).first())
        tprops = tile.toDictionary().getInfo()

        # Create the composite for one year
        imgs = lxtools.sr_collection(
            tile.geometry(),
            str(year), str(year+1), startdoy, enddoy,
            **coll_kwargs
        )
        imgs = imgs.map(lambda i: lxtools.specixs(i, ixlist=['ndvi', 'nbr']))
        img = imgs.median()
        
        # Cast to Int16 for a smaller export
        img = img.multiply(1000).toInt16()

        # Set the nodata values, otherwise they'll be 0, which is a valid value
        img = img.unmask(-32768)

        # Setup for exporting the exact tile extent and resolution.
        outname = 'ls_'+hv+'_'+str(year)
        crs='epsg:5070'
        scale = 30.0
        dimx = int((tprops['maxx'] - tprops['minx'])/scale)
        dimy = int((tprops['maxy'] - tprops['miny'])/scale)
        dims = str(dimx)+'x'+str(dimy)
        transform = [scale, 0.0, float(tprops['minx']), 0.0, -scale, float(tprops['maxy'])]

        # Make sure a single file is used
        shardSize = 256
        fileDimensions = (int(np.ceil(dimx / shardSize) * shardSize), int(np.ceil(dimy / shardSize) * shardSize))
        nbands = (endy-starty+1) * len(coll_kwargs['bands'])

        # to drive
        task = ee.batch.Export.image.toDrive(
            image=img, 
            description=outname,
            fileNamePrefix=outname,
            folder = "gee",
            dimensions=dims,
            crs=crs,
            crsTransform=str(transform),
            maxPixels=float(dimx)*dimy*nbands,
            fileDimensions=fileDimensions
        )

        task_list.append(task)
        task.start()

In [ ]:
statuses = check_tasks(task_list)

In [ ]:
# Minutes to complete
(statuses[-1]['update_timestamp_ms'] - statuses[0]['creation_timestamp_ms']) / 1000 / 60

### Stacked - one tile, all years
This method is faster to export (~7 min per tile, ~9 min overall with multiple tasks running). However, the images from annual export need to be stacked into a single image and then unstacked after export, which also takes some time. 

In general for annual composites, it seems that exporting all bands and years in a single image tends to be much faster. The more bands and years you're exporting the better this method is in comparison to single year exports. However, GEE has a limit of 16 GB uncompressed for a single image export task, which is 320 bands in 16-bit image for a 5k x 5k ARD tile. Note that runtimes depend on the google server load, so they can vary quite a bit.

In [ ]:
# Loop over the tiles to create the composite collection and export one image for each tile
# with years and bands stacked for faster export (takes ~7 minutes per tile)
task_list = []
for hv in tilehvs:
    print(hv)
    tile = ee.Feature(tilesfc.filterMetadata('hv', 'equals', hv).first())
    tprops = tile.toDictionary().getInfo()
    
    # Create the annual composite collection
    comps = ts.annual_composites(
        tile.geometry(),
        starty, endy, startdoy, enddoy,
        lxtools.sr_collection, 
        median_specix,
        coll_kwargs,
        fill=True
    )
    
    # Stack the images into bands of year_band for faster export
    comps = comps.sort('sytem:time_start')
    img = ts.stack_annual(comps) # stack annual depends on the images having a 'year'bb property
    
    # Cast to Int16 for a smaller export
    img = img.multiply(1000).toInt16()
    
    # Set the nodata values, otherwise they'll be 0, which is a valid value
    img = img.unmask(-32768)
    
    # Setup for exporting the exact tile extent and resolution.
    outname = 'ls_'+hv
    crs='epsg:5070'
    scale = 30.0
    dimx = int((tprops['maxx'] - tprops['minx'])/scale)
    dimy = int((tprops['maxy'] - tprops['miny'])/scale)
    dims = str(dimx)+'x'+str(dimy)
    transform = [scale, 0.0, float(tprops['minx']), 0.0, -scale, float(tprops['maxy'])]
    
    # Make sure a single file is used
    shardSize = 256
    fileDimensions = (int(np.ceil(dimx / shardSize) * shardSize), int(np.ceil(dimy / shardSize) * shardSize))
    nbands = (endy-starty+1) * len(coll_kwargs['bands'])

    # to drive
    task = ee.batch.Export.image.toDrive(
        image=img, 
        description=outname,
        fileNamePrefix=outname,
        folder = "gee",
        dimensions=dims,
        crs=crs,
        crsTransform=str(transform),
        maxPixels=float(dimx)*dimy*nbands,
        fileDimensions=fileDimensions
    )
    
    task_list.append(task)
    task.start()

In [ ]:
statuses = check_tasks(task_list)

In [ ]:
# minutes from start to end
(statuses[-1]['update_timestamp_ms'] - statuses[0]['creation_timestamp_ms']) / 1000 / 60

### Sorting Stacked Images
If you export an image stacked with bands labeled as "year_bandname", you may want to reorganize them after export. I will typically organize images into a "tile" folder with subfolders for each year and all the spectral bands stored in each image. This makes it easy to create virtual mosaics of all the tiles in a single year subfolder and then visualize multiple spectral bands in GIS software.

Since GEE exports images as pixel-interleaved not band-interleaved, reading in images with many bands can be very slow, but it's still typically much faster than exporting many separate images from GEE.

In [ ]:
# Setup the input and output folder and parameters
# Make sure that your annual composite tiles are the only images in your GEE folder or move them to a separate folder
indir = r"E:\My Drive\gee"
paths = glob(os.path.join(indir, "*.tif"))

outdir = r"C:\scratch"
temp="{base}_{tile}.tif"
multi = None
nodata = -32768
basename='ls'

In [ ]:
# run sorting
ts.split_stack(paths, outdir, basename, temp=temp, nodata=nodata, multi=multi, threads=8)

# LandTrendr Time Series
An image collection like above can also be run through the LandTrendr temporal segmentation algorithm prior to export to provide 'fitted' estimates of spectral reflectance that reduce year to year variations that are likely not do to on the ground changes.

The 'lt_fitted' function runs an annual image collection through Landtrendr and fits each band independently by default, but a single band can also be used for determining the vertices with remaining bands fit to that temporal segmentation. Additional keyword arguments for LandTrendr parameters are passed on to ee.Algorithms.TemporalSegmentation.LandTrendr.

The below example shows how to export a set of tiles run through LandTrendr with two files having different spectral bands or indices exported for each tile. Each band and spectral index is temporally segmented independently. 

Splitting the tile export into multiple files with different bands may be necessary to stay under the 16GB uncompressed export limit. This is true whether you're exporting a regular annual composite time series or LandTrendr time series.

## Set up

In [ ]:
# Filter for ARD tiles intersecting the AOI
aoi = ee.FeatureCollection('TIGER/2018/Counties').filterMetadata('NAME', 'equals', 'Larimer')
tilesfc = ee.FeatureCollection("users/stevenf/other/conus_ard_grid_coords")
tilesfc = tilesfc.filterBounds(aoi)

# Get a list of tiles to loop over
tilehvs = tilesfc.aggregate_array('hv').getInfo()
tilehvs

In [ ]:
# Specify annual composite collection parameters

# time filtering
# In this example we only use 10 years, but you'd typically want to run LandTrendr on the full Landsat time series (1984-present)
starty = 2010
endy = 2019 
startdoy = 152   # June 1st
enddoy = 273     # Sept 30th

# TDOM set up
TDOMStats = ee.ImageCollection('projects/lcms-tcc-shared/assets/CS-TDOM-Stats/TDOM').mosaic()
mean_img = TDOMStats.select(['Landsat_nir_mean', 'Landsat_swir1_mean'], ['nir', 'swir1']) 
stddev_img = TDOMStats.select(['Landsat_nir_stdDev', 'Landsat_swir1_stdDev'], ['nir', 'swir1'])
tdom_kwargs = {'mean_img':mean_img, 'stddev_img':stddev_img, 'sum_thresh':3500}

# Masking differences from defaults
mask_kwargs = {
    'water':None
}

# Export different sets of bands in multiple files
bdict = {'orig':['red', 'nir', 'swir1', 'swir2'],
         'spix':['ndvi', 'nbr', 'ndmi']}

# Collection options
coll_kwargs = {
    'rescale':False,
    'bands':bdict['orig'], # At least nir and swir1 are needed for TDOM
    'cloud_cover':50,
    'tdom':True,
    'mask_kwargs':mask_kwargs,
    'tdom_kwargs':tdom_kwargs,
    'exclude': ['LE07_033032_20180612', 'LC08_034031_20180729']
}

# LandTrendr parameters
lt_kwargs = { 
  'maxSegments':            6,
  'spikeThreshold':         0.9,
  'vertexCountOvershoot':   3,
  'preventOneYearRecovery': True,
  'recoveryThreshold':      0.25,
  'pvalThreshold':          0.05,
  'bestModelProportion':    0.75, # LCMS uses 1.25
  'minObservationsNeeded':  6
}

## Export
Run the export process just like with the annual mosaics, but use the medoid composite and calculate spectral indices from the medoid image.  
This should take about 25 minutes to complete per image and 50 minutes total depending on server load and how many tasks run concurrently.

In [ ]:
# Export LT fitted time series for each tile with original bands and spectral indices in separate files
task_list = []
for hv in tilehvs:
    tile = ee.Feature(tilesfc.filterMetadata('hv', 'equals', hv).first())
    tprops = tile.toDictionary().getInfo()
    
    # Create the annual composite collection
    comps = ts.annual_composites(
        tile.geometry(),
        starty, endy, startdoy, enddoy,
        lxtools.sr_collection, 
        ts.medoid,
        coll_kwargs,
        fill=True       # Filling years with no images with an empty composite image is necessary for LandTrendr
    )  
    
    for k, bands in bdict.items():
        print(hv, k)
        if k=='spix':
            # Get spectral indices and convert to int16 range
            # Note that calculating spectral indices on the composite is fine since medoid was used.
            # If using median, spectral indices should be calculated for each image prior to compositing.
            comps = comps.map(lambda i: (lxtools.specixs(i, ixlist=bands)
                                                .multiply(1000)
                                                .copyProperties(i, i.propertyNames())))
        
        # Run LT and extract fit
        imgs_fit = ts.lt_fitted(comps, flip_bands=True, fit_band=None, **lt_kwargs)
    
        # Stack the images into bands of year_band for faster export
        imgs_fit = imgs_fit.sort('sytem:time_start')
        img = ts.stack_annual(imgs_fit) # stack annual depends on the images having a 'year'bb property
    
        # Cast to Int16 for a smaller export
        img = img.toInt16()

        # Set the nodata values, otherwise they'll be 0, which is a valid value
        img = img.unmask(-32768)

        # Setup for exporting the exact tile extent and resolution.
        outname = 'lt_'+hv+'_'+k
        crs='epsg:5070'
        scale = 30.0
        dimx = int((tprops['maxx'] - tprops['minx'])/scale)
        dimy = int((tprops['maxy'] - tprops['miny'])/scale)
        dims = str(dimx)+'x'+str(dimy)
        transform = [scale, 0.0, float(tprops['minx']), 0.0, -scale, float(tprops['maxy'])]

        # Make sure a single file is used
        shardSize = 256
        fileDimensions = (int(np.ceil(dimx / shardSize) * shardSize), int(np.ceil(dimy / shardSize) * shardSize))
        nbands = (endy-starty+1) * len(coll_kwargs['bands'])

        # to drive
        task = ee.batch.Export.image.toDrive(
            image=img, 
            description=outname,
            fileNamePrefix=outname,
            folder = "gee",
            dimensions=dims,
            crs=crs,
            crsTransform=str(transform),
            maxPixels=float(dimx)*dimy*nbands,
            fileDimensions=fileDimensions
        )

        task_list.append(task)
        task.start()

In [ ]:
statuses = check_tasks(task_list)

In [ ]:
# minutes from start to end
(statuses[-1]['update_timestamp_ms'] - statuses[0]['creation_timestamp_ms']) / 1000 / 60

## Sort LT Stacked Images
This is the same as sorting the annual composites above, but now we have bands split into multiple files for each tile ('orig' and 'spix'). So the parameters and template need to be set up to organize these bands together into a single file per year.

Make sure the LT fitted images just exported are the only files in 'indir' before sorting.

In [ ]:
# Setup the input and output folder and parameters
# Make sure that your annual composite tiles are the only images in your GEE folder or move them to a separate folder
indir = r"E:\My Drive\gee"
paths = glob(os.path.join(indir, "*.tif"))

outdir = r"G:\tiles"
os.makedirs(outdir, exist_ok=True)
temp="{base}_{tile}_{multi}.tif"
multi = ['orig', 'spix']
nodata = -32768
basename='lt'

In [ ]:
# run sorting
ts.split_stack(paths, outdir, basename, temp=temp, nodata=nodata, multi=multi, threads=8)

# Mosaic Tiles
Either annual composites or Landtrendr annual time series exported as tiles can be mosaiced together into a single virtual image (VRT). Making a virtual image is fast and avoids duplicating the data. VRTs are just XML files that describe what files GIS software should look for when pulling up an image in a certain area.  You can use input paths relative (like below) to the output VRT to be able to move both the tiles and the virtual mosaics together and have the mosaics still work.  
This snippet uses gdalbuildvrt which requires gdal to be installed in your current python environment. 

In [ ]:
mosaic_dir = r"G:\mosaics"
tile_dir = r"G:\tiles"

years = os.listdir(tile_dir)
os.makedirs(mosaic_dir, exist_ok=True)

# don't specify an absolute for the VRT if needing relative paths to the tiffs
cwd = os.getcwd()
os.chdir(mosaic_dir)

for y in years:
    # create vrt
    outname = "lt_larimer_"+y+".vrt"
    paths = glob(os.path.join(tile_dir, y, "*.tif"))
    paths = [os.path.relpath(p, mosaic_dir) for p in paths]
    paths.sort()
    cmd = "gdalbuildvrt " + outname + " " + " ".join(paths)
    stdout = subprocess.check_output(cmd)

    # set band descriptions
    with rasterio.open(paths[0]) as src:
        descs = src.descriptions
    with rasterio.open(outname, 'r+') as src:
        src.descriptions = descs
        
os.chdir(cwd)

In [ ]:
print(os.listdir(mosaic_dir))